<a href="https://colab.research.google.com/github/Ishant-93/Artificial-Intelligence/blob/main/fine_tuning_spam.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd 
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/SMSSpamCollection', sep='\t', names=["label", "message"])
df.head()

In [ ]:
#independent variables
X=list(df['message'])

#dependent variables
Y=list(df['label'])

In [ ]:
Y=list(pd.get_dummies(Y, drop_first=True, dtype=int)['spam'])

In [ ]:
Y

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=0)

In [ ]:
pip install transformers


In [ ]:
from transformers import DistilBertTokenizerFast
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

In [ ]:
train_encodings = tokenizer(X_train, truncation=True, padding=True)
test_encodings = tokenizer(X_test, truncation=True, padding=True)


In [ ]:
import tensorflow as tf

train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    Y_train
))

test_dataset = tf.data.Dataset.from_tensor_slices((
    dict(test_encodings),
    Y_test
))


In [ ]:
train_dataset

In [ ]:
from transformers import TFDistilBertForSequenceClassification, TFTrainer, TFTrainingArguments

training_args = TFTrainingArguments(
    output_dir='.\results',
    num_train_epochs=2,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=16,
    warmup_steps= 500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    eval_steps=100
)

In [ ]:
with training_args.strategy.scope():
  model = TFDistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased")

trainer = TFTrainer(
    model = model,
    args = training_args,
    train_dataset = train_dataset,
    eval_dataset=test_dataset
)

In [ ]:
trainer.train()

In [ ]:
trainer.evaluate()

In [ ]:
trainer.predict(test_dataset)
##output of this is a tuple.

In [ ]:
##main prediction.
trainer.predict(test_dataset)[1]
